In [59]:
import numpy as np
import pandas as pd
import time

In [48]:
from collections import Counter
from itertools import combinations, groupby

In [2]:
orders=pd.read_csv('C:/Users/Nikhar/Downloads/Side_work/Instacart/Data/order_products__prior.csv')

In [62]:
products=pd.read_csv('C:/Users/Nikhar/Downloads/Side_work/Instacart/Data/products.csv')

In [3]:
orders.shape

(32434489, 4)

In [4]:
orders.head()

order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0

In [6]:
#using the order as a series
orders=orders.set_index('order_id')['product_id']

In [13]:
print("total no. of orders")
len(orders.index.unique())

total no. of orders


3214874

In [12]:
print("total unique products sold")
len(orders.value_counts())

total unique products sold


49677

In [7]:
orders.head(10)

order_id
2    33120
2    28985
2     9327
2    45918
2    30035
2    17794
2    40141
2     1819
2    43668
3    33754
Name: product_id, dtype: int64

In [15]:
products.head()

product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   
3           4  Smart Ones Classic Favorites Mini Rigatoni Wit...        38   
4           5                          Green Chile Anytime Sauce         5   

   department_id  
0             19  
1             13  
2              7  
3              1  
4             13

In [61]:
#frequency count for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")
    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))

#generating the item pairs
def get_item_pairs(order_item):
    order_item=order_item.reset_index().as_matrix()
    for order_id, order_object in groupby(order_item, lambda X:X[0]  ):
        item_list= [item[1] for item in order_object]
        
        for item_pair in combinations(item_list,2):
            yield item_pair
    
def merge_item_stats(item_pairs,item_stats):
    return (item_pairs.merge(item_stats.rename(columns={'freq':'freqA','support':'supportA'}),left_on='item_A', right_index=True).merge(item_stats.rename(columns={'freq':'freqB','support':'supportB'}),left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns] 


In [57]:
#calculating the association rules:
def association_rule(orders_product,min_support):
    print("starting/total product: {:22}".format(len(orders)))
    #item frequency and support
    item_stats= freq(orders_product).to_frame('freq')
    #unique count of orders
    #
    item_stats['support']= item_stats['freq']/order_count(orders_product)*100
    
    #filtering items below min support
    qualifying_product=item_stats[item_stats['support']>=min_support].index
    #taking only the product order combination with qualifying product with min support
    orders_product=orders_product[orders_product.isin(qualifying_product)]
    
    print("products with support >={}:{:15d}".format(min_support, len(qualifying_product)))
    print("Remainig products: {:21}".format(len(orders_product)))
#     print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    
    #filtering out order items with less than 2 products
    order_size= freq(orders_product.index)
    qualifying_products=order_size[order_size>=2].index
    orders_product=orders_product[orders_product.index.isin(qualifying_products)]
    
    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_products)))
    print("Remaining order_product: {:21d}".format(len(orders_product)))
    
    #recalculating item/product frequenchy and support
    item_stats = freq(orders_product).to_frame('freq')
    item_stats['support']= item_stats['freq']/order_count(orders_product)*100
    
    #retrieving item/product pairs
    item_pair_gen= get_item_pairs(orders_product)
    
    #calculating item pairs frequency and support
    item_pairs= freq(item_pair_gen).to_frame('freqAB')
    item_pairs['supportAB']= item_pairs['freqAB']/len(qualifying_products)*100
    
    print("Product pairs: {:31d}".format(len(item_pairs)))
    
    #Filter out item pairs with support below min support
    item_pairs= item_pairs[item_pairs['supportAB']>=min_support]
    print('item pair with support >= {}:{:10d}\n'.format(min_support,len(item_pairs)))
    
    #Association rules and matrices
    item_pairs= item_pairs.reset_index().rename(columns={'level_0':'item_A','level_1':'item_B'})
    item_pairs= merge_item_stats(item_pairs,item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    #returning association rules sorted by lift in descending order
    return item_pairs.sort_values('lift',ascending=False)
    

In [60]:
%%time
print(time.time())
rules=association_rule(orders,0.01)
print(time.time())

1537595412.0797608
starting/total product:               32434489
products with support >=0.01:          10906
Remainig products:              29843570
Remaining orders with 2+ items:     3013325
Remaining order_product:              29662716


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


Product pairs:                        30622410
item pair with support >= 0.01:     48751

1537596014.574105
Wall time: 10min 2s


In [65]:
item_name   = products.rename(columns={'product_id':'item_id', 'product_name':'item_name'})
rules_final = merge_item_name(rules, item_name).sort_values('lift', ascending=False)
display(rules_final)

itemA  \
0       Organic Strawberry Chia Lowfat 2% Cottage Cheese   
1                    Grain Free Chicken Formula Cat Food   
3              Organic Fruit Yogurt Smoothie Mixed Berry   
9      Nonfat Strawberry With Fruit On The Bottom Gre...   
10        Organic Grapefruit Ginger Sparkling Yerba Mate   
11      Baby Food Pouch - Roasted Carrot Spinach & Beans   
12       Unsweetened Whole Milk Mixed Berry Greek Yogurt   
23                           Uncured Cracked Pepper Beef   
24                                  Organic Mango Yogurt   
2                    Grain Free Chicken Formula Cat Food   
25                               Raspberry Essence Water   
13              Unsweetened Whole Milk Strawberry Yogurt   
14             Unsweetened Whole Milk Peach Greek Yogurt   
44     Oh My Yog! Pacific Coast Strawberry Trilayer Y...   
55     Mighty 4 Kale, Strawberry, Amaranth & Greek Yo...   
20             Unsweetened Whole Milk Peach Greek Yogurt   
65              0% Greek, Blueberry on the Bottom Yogurt   
15       Unsweetened Whole Milk Mixed Berry Greek Yogurt   
43             Unsweetened Whole Milk Peach Greek Yogurt   
26                          Unsweetened Blackberry Water   
19       Unsweetened Whole Milk Mixed Berry Greek Yogurt   
16              Unsweetened Whole Milk Strawberry Yogurt   
56     Mighty 4 Sweet Potato, Blueberry, Millet & Gre...   
74                                Sweet Potatoes Stage 2   
79                                     Compostable Forks   
75                     Organic Stage 2 Carrots Baby Food   
42              Unsweetened Whole Milk Strawberry Yogurt   
21         Unsweetened Whole Milk Blueberry Greek Yogurt   
17         Unsweetened Whole Milk Blueberry Greek Yogurt   
83                            Cream Top Blueberry Yogurt   
...                                                  ...   
22444                                        Large Lemon   
2577                                           Red Onion   
250                              Roasted Pine Nut Hummus   
655                        Organic Large Green Asparagus   
40897                                             Banana   
2652                                             Spinach   
2722                                          Sour Cream   
11143                                Organic Blueberries   
2537                                        Green Onions   
1386                                 2% Reduced Fat Milk   
3291                                 2% Reduced Fat Milk   
530                               Chocolate Chip Cookies   
10681                                        Half & Half   
5446                         Organic Reduced Fat 2% Milk   
11455                                             Banana   
11421                             Bag of Organic Bananas   
2568                 Asparation/Broccolini/Baby Broccoli   
19596                                             Banana   
2319                                    Fridge Pack Cola   
11017                               Organic Baby Spinach   
22572                                Organic Raspberries   
11012                               Organic Strawberries   
246                                                 Soda   
11555                                             Banana   
1474                                        Strawberries   
7271                                Organic Strawberries   
6763                                Organic Hass Avocado   
4387                                     Organic Avocado   
2596                                              Banana   
670                               Bag of Organic Bananas   

                                                   itemB  freqAB  supportAB  \
0             Organic Cottage Cheese Blueberry Acai Chia     306   0.010155   
1                     Grain Free Turkey Formula Cat Food     318   0.010553   
3                  Apple Blueberry Fruit Yogurt Smoothie     349   0.011582   
9             